In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

In [2]:
path = r'C:\Users\msyeu\Instacart Basket Analysis'

In [3]:
df_orders_products_merged = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'orders_products_merged.pkl'))

In [4]:
df_orders_products_merged.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,both
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23,19,4.4,both
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,both
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both


In [5]:
df = df_orders_products_merged[:1000000]

In [6]:
df.shape

(1000000, 15)

In [7]:
#Define a function

def price_label(row):
    
    if row['prices'] <= 5:
        return 'Low range product'
    elif (row['prices'] > 5) and (row['prices'] <= 15):
        return 'Mid range product'
    elif row['prices'] >15:
        return 'High range product'
    else: return 'Not enough data'

In [8]:
df['price_range'] = df.apply(price_label, axis=1)

C:\Users\msyeu\AppData\Local\Temp\ipykernel_32192\2848743372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [9]:
df['price_range'].value_counts(dropna = False)

price_range
Mid range product     673183
Low range product     314392
High range product     12425
Name: count, dtype: int64

2. If-statement using loc()

In [12]:
df_orders_products_merged.loc[df_orders_products_merged['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [13]:
df_orders_products_merged.loc[(df_orders_products_merged['prices'] <= 15) & (df_orders_products_merged['prices'] >5), 'price_range_loc'] = 'Mid-range product'

In [15]:
df_orders_products_merged.loc[df_orders_products_merged['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [16]:
df_orders_products_merged['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     16134940
Low-range product      7475149
High-range product      308106
Name: count, dtype: int64

In [17]:
df_orders_products_merged['order_dow'].value_counts(dropna = False)

order_dow
0    4574752
1    4177805
6    3318571
2    3107781
5    3105424
3    2840875
4    2792987
Name: count, dtype: int64

In [18]:
result = []

In [19]:
for value in df_orders_products_merged["order_dow"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [20]:
df_orders_products_merged['busiest_day'] = result

In [21]:
df_orders_products_merged['busiest_day'].value_counts(dropna = False)

busiest_day
Regularly busy    16550456
Busiest day        4574752
Least busy         2792987
Name: count, dtype: int64

In [22]:
len(result)

23918195

# 2. Suppose your clients have changed their minds about the labels you created in your “busiest_day” column. Now, they want “Busiest day” to become “Busiest days” (plural).This label should correspond with the two busiest days of the week as opposed to the single busiest day. At the same time, they’d also like to know the two slowest days. Create a new column for this using a suitable method.


In [23]:
labeling_counts = df_orders_products_merged['order_dow'].value_counts(dropna = False)

In [24]:
labeling_counts

order_dow
0    4574752
1    4177805
6    3318571
2    3107781
5    3105424
3    2840875
4    2792987
Name: count, dtype: int64

In [25]:
labeling_counts.index

Index([0, 1, 6, 2, 5, 3, 4], dtype='int64', name='order_dow')

In [26]:
day_number = list(labeling_counts.index)

In [27]:
day_number

[0, 1, 6, 2, 5, 3, 4]

In [28]:
results = []

In [29]:
for value in df_orders_products_merged['order_dow']:
    if value == day_number[0] or value == day_number[1]:
        results.append('Busiest days')
    elif value == day_number[len(day_number) - 1] or value == day_number[len(day_number) - 2]:
        results.append('Slowest days')
    else:
        results.append('Regulary busy days')

In [30]:
df_orders_products_merged['busiest_days'] = results

# Step 3 - Check the values of this new column for accuracy.

In [31]:
df_orders_products_merged['busiest_days'].value_counts(dropna = False)

busiest_days
Regulary busy days    9531776
Busiest days          8752557
Slowest days          5633862
Name: count, dtype: int64

# Step 4 When too many users make Instacart orders at the same time, the app freezes. The senior technical officer at Instacart wants you to identify the busiest hours of the day. Rather than by hour, they want periods of time labeled “Most orders,” “Average orders,” and “Fewest orders.” Create a new column containing these labels called “busiest_period_of_day.”

In [32]:
#Converting frequency into dictionary
dict_labeling_counts = df_orders_products_merged['order_hour_of_day'].value_counts(dropna = False).to_dict()

In [33]:
dict_labeling_counts

{10: 2036438,
 11: 2019440,
 14: 1985727,
 13: 1964787,
 15: 1963384,
 12: 1933480,
 16: 1868518,
 9: 1810519,
 17: 1536352,
 8: 1268701,
 18: 1207264,
 19: 929401,
 20: 722233,
 7: 658825,
 21: 593153,
 22: 471264,
 23: 297495,
 6: 213047,
 0: 161914,
 1: 84061,
 5: 64788,
 2: 51234,
 4: 38923,
 3: 37247}

In [34]:
# I used percentile to group most, average and fewest orders. 
list_quantile = list(df_orders_products_merged['order_hour_of_day'].value_counts(dropna = False).quantile([0.75, 0.5, 0.25]))

In [35]:
list_quantile

[1884758.5, 825817.0, 200263.75]

In [36]:
results_labeling_hour = []

In [37]:
for value in df_orders_products_merged['order_hour_of_day']:
    for key, val in dict_labeling_counts.items():
        if value == key:
            if val < list_quantile[2]:
                results_labeling_hour.append('Fewest orders')
            elif val >= list_quantile[2] and val < list_quantile[1]:
                results_labeling_hour.append('Average orders')
            elif val >= list_quantile[1]:
                results_labeling_hour.append('Most orders')

In [38]:
def labeling_result (x):
    most_orders = x.count('Most orders')
    average_orders = x.count('Average orders')
    fewest_orders = x.count('Fewest orders')
    
    print(f'Most orders: {most_orders}')
    print(f'Average orders: {average_orders}')
    print(f'Fewest orders: {fewest_orders}')
    print(f'Total: {most_orders + average_orders + fewest_orders}')

In [39]:
labeling_result(results_labeling_hour)

Most orders: 20524011
Average orders: 2956017
Fewest orders: 438167
Total: 23918195


In [40]:
df_orders_products_merged['order_hour_of_day'].shape

(23918195,)

In [41]:
df_orders_products_merged['busiest_period_of_day'] = results_labeling_hour

# Step 5. Print the frequency for this new column

In [42]:
df_orders_products_merged['busiest_period_of_day'].value_counts(dropna = False)

busiest_period_of_day
Most orders       20524011
Average orders     2956017
Fewest orders       438167
Name: count, dtype: int64

In [44]:
#Export your data
df_orders_products_merged.to_pickle(os.path.join(path, 'Data', 'Prepared Data', 'orders_products_merged_update.pkl'))